In [17]:
#task 1
import cv2
import numpy as np

# Load the video file
video_path = "C:\\Users\\DELL\\Downloads\\gettyimages-1995814914-640_adpp.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize the background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=300, varThreshold=50, detectShadows=True)

# Variables for tracking
tracking_points = []
MAX_TRACK_HISTORY = 10

# Parameters for filtering
MIN_CONTOUR_AREA = 1500  # Minimum area to consider (adjust as needed)
MIN_ASPECT_RATIO = 0.4   # Minimum aspect ratio (height/width) for a person
MAX_ASPECT_RATIO = 2.5   # Maximum aspect ratio for a person

def process_frame(frame):
    # Step 1: Apply background subtraction to get the foreground mask
    fg_mask = bg_subtractor.apply(frame)
    
    # Step 2: Remove noise with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)
    
    # Step 3: Find contours in the mask
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Step 4: Filter contours to detect the person based on area and aspect ratio
    person_contour = None
    max_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > MIN_CONTOUR_AREA:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = h / float(w)

            # Check if contour meets size and shape criteria for a person
            if MIN_ASPECT_RATIO < aspect_ratio < MAX_ASPECT_RATIO:
                if area > max_area:
                    max_area = area
                    person_contour = contour

    # Step 5: Draw bounding box around the largest valid contour if found
    if person_contour is not None:
        x, y, w, h = cv2.boundingRect(person_contour)
        center = (x + w // 2, y + h // 2)
        
        # Save center point for smooth tracking
        tracking_points.append(center)
        if len(tracking_points) > MAX_TRACK_HISTORY:
            tracking_points.pop(0)
        
        # Draw bounding box around detected person
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
    # Optional: Draw tracking path to show recent movement
    for i in range(1, len(tracking_points)):
        if tracking_points[i - 1] is None or tracking_points[i] is None:
            continue
        thickness = int(np.sqrt(MAX_TRACK_HISTORY / float(i + 1)) * 2.5)
        cv2.line(frame, tracking_points[i - 1], tracking_points[i], (0, 0, 255), thickness)

    return frame

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the current frame
    processed_frame = process_frame(frame)

    # Display the processed frame
    cv2.imshow("Person Tracking", processed_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

In [21]:

#Task 5
import cv2
import numpy as np
from collections import defaultdict

# Path to your video file
video_path = "C:\\Users\\DELL\\Downloads\\gettyimages-671426188-640_adpp.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=700, varThreshold=50, detectShadows=False)

# Frames per second of the video to calculate time per frame
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1 / fps  # Time per frame in seconds

# Dictionary to store dwell time for each unique object
dwell_times = defaultdict(float)
object_centers = {}  # Dictionary to store the last known position of each object

object_id_count = 0  # Counter to assign unique IDs to each object

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgmask = fgbg.apply(frame)

    # Additional filtering to clean up the foreground mask
    fgmask = cv2.medianBlur(fgmask, 5)  # Reduce noise with median blur

    # Threshold to create binary image, fine-tune based on your scene
    _, thresh = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    current_frame_centers = []  # Track the current frame's object centers

    for cnt in contours:
        # Filter out small contours
        if cv2.contourArea(cnt) < 1000:  # Adjust contour area threshold if needed
            continue

        # Get bounding box for each object
        x, y, w, h = cv2.boundingRect(cnt)
        center = (x + w // 2, y + h // 2)

        # Check if the object matches any previous object based on proximity
        matched_object_id = None
        for obj_id, prev_center in object_centers.items():
            if np.linalg.norm(np.array(center) - np.array(prev_center)) < 50:  # Adjust distance threshold if needed
                matched_object_id = obj_id
                break

        # If no match found, assign a new ID
        if matched_object_id is None:
            matched_object_id = object_id_count
            object_id_count += 1

        # Update the center position of the matched object
        object_centers[matched_object_id] = center
        current_frame_centers.append(matched_object_id)

        # Increment the dwell time for this object by the time per frame
        dwell_times[matched_object_id] += frame_time

        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {matched_object_id} Time: {dwell_times[matched_object_id]:.2f}s", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Remove objects that are no longer detected in the frame
    object_centers = {obj_id: center for obj_id, center in object_centers.items() if obj_id in current_frame_centers}

    # Display the frame with detections
    cv2.imshow("Dwell Time Detection", frame)

    # Exit condition
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Print the total dwell time for each object/person detected in the video
print("Total Dwell Time for Each Detected Object:")
for obj_id, time in dwell_times.items():
    print(f"Object ID {obj_id}: {time:.2f} seconds")

Total Dwell Time for Each Detected Object:
Object ID 0: 0.03 seconds
Object ID 1: 3.54 seconds
Object ID 2: 3.04 seconds
Object ID 3: 0.13 seconds
Object ID 4: 0.07 seconds
Object ID 5: 0.63 seconds
Object ID 6: 0.10 seconds
Object ID 7: 0.37 seconds
Object ID 8: 3.07 seconds
Object ID 9: 0.10 seconds
Object ID 10: 0.23 seconds
Object ID 11: 0.20 seconds
Object ID 12: 2.00 seconds
Object ID 13: 0.10 seconds
Object ID 14: 0.27 seconds
Object ID 15: 0.23 seconds
Object ID 16: 0.20 seconds
Object ID 17: 0.13 seconds
Object ID 18: 0.07 seconds
Object ID 19: 0.07 seconds
Object ID 20: 0.23 seconds
Object ID 21: 0.20 seconds
Object ID 22: 0.07 seconds
Object ID 23: 0.20 seconds
Object ID 24: 0.40 seconds
Object ID 25: 0.07 seconds
Object ID 26: 0.07 seconds
Object ID 27: 0.07 seconds
Object ID 28: 0.53 seconds
Object ID 29: 0.30 seconds
Object ID 30: 0.03 seconds
Object ID 31: 0.63 seconds
Object ID 32: 0.63 seconds
Object ID 33: 0.03 seconds
Object ID 34: 0.03 seconds
Object ID 35: 0.13 sec

In [22]:
#task 3
import cv2
import numpy as np
import os


# Set paths for reference image and video
reference_image_path = "C:\\Users\\DELL\\Downloads\\Screenshot 2024-11-14 122059.png"
video_path = "C:\\Users\\DELL\\Downloads\\Elevator - Racism. It Stops With Me.mp4"

# Specify output folder for matched frames
output_folder = "C:\\Users\\DELL\\Desktop\\output"
os.makedirs(output_folder, exist_ok=True)

# Load the reference image and convert it to grayscale
reference_image = cv2.imread(reference_image_path)
reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Detect face in the reference image
ref_faces = face_cascade.detectMultiScale(reference_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Ensure there's at least one face detected in the reference image
if len(ref_faces) == 0:
    print("No faces found in the reference image.")
    exit()
else:
    print(f"{len(ref_faces)} face(s) detected in the reference image.")

# Extract the detected face from the reference image (use the largest face)
x, y, w, h = max(ref_faces, key=lambda face: face[2] * face[3])
reference_face = reference_gray[y:y+h, x:x+w]

# Load the video
cap = cv2.VideoCapture(video_path)

# Initialize a frame counter
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the current frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    match_found = False  # Flag to check if any match is found in the current frame

    # Process each detected face in the frame
    for (fx, fy, fw, fh) in faces:
        # Extract the face from the frame
        face_in_frame = gray_frame[fy:fy+fh, fx:fx+fw]

        # Resize the reference face and detected face to the same size for comparison
        resized_reference = cv2.resize(reference_face, (fw, fh))
        match_result = cv2.matchTemplate(face_in_frame, resized_reference, cv2.TM_CCOEFF_NORMED)
        _, match_val, _, _ = cv2.minMaxLoc(match_result)

        # Check if match value exceeds threshold (indicates a match)
        match_threshold = 0.7
        if match_val > match_threshold:
            # Draw a rectangle around the matching face in the frame
            cv2.rectangle(frame, (fx, fy), (fx + fw, fy + fh), (0, 255, 0), 2)
            cv2.putText(frame, f'Match: {match_val:.2f}', (fx, fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print(f"Match found in frame {frame_count} with similarity score: {match_val:.2f}")

            # Save the frame with match to the output folder
            output_frame_path = os.path.join(output_folder, f'frame_{frame_count}.jpg')
            cv2.imwrite(output_frame_path, frame)
            match_found = True

    # Display only frames with a detected match
    if match_found:
        cv2.imshow('Matching Frame', frame)

    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()



1 face(s) detected in the reference image.
Match found in frame 107 with similarity score: 0.77
Match found in frame 108 with similarity score: 0.82
Match found in frame 109 with similarity score: 0.85
Match found in frame 110 with similarity score: 0.87
Match found in frame 111 with similarity score: 0.85
Match found in frame 112 with similarity score: 0.87
Match found in frame 113 with similarity score: 0.89
Match found in frame 114 with similarity score: 0.90
Match found in frame 115 with similarity score: 0.96
Match found in frame 116 with similarity score: 1.00
Match found in frame 117 with similarity score: 0.95
Match found in frame 118 with similarity score: 0.97
Match found in frame 119 with similarity score: 0.87
Match found in frame 120 with similarity score: 0.89
Match found in frame 121 with similarity score: 0.87
Match found in frame 122 with similarity score: 0.80
Match found in frame 124 with similarity score: 0.71


In [25]:
#task 6
import cv2
import numpy as np

# Path to your video file
video_path = r"C:\\Users\\DELL\\Downloads\\20653316-uhd_2160_3840_30fps.mp4"  # Replace with the path to your video
cap = cv2.VideoCapture(video_path)

# Initialize background subtractor for motion detection
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Define range for white color in HSV
lower_white = np.array([0, 0, 200], dtype=np.uint8)
upper_white = np.array([180, 25, 255], dtype=np.uint8)

# Counter for white cars
white_car_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame for consistent processing
    frame = cv2.resize(frame, (800, 600))

    # Apply background subtraction to get the moving parts (cars)
    fg_mask = bg_subtractor.apply(frame)

    # Filter out shadows by thresholding
    _, fg_mask = cv2.threshold(fg_mask, 100, 255, cv2.THRESH_BINARY)

    # Find contours of the moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 900:  # Adjust contour area threshold as needed
            # Get bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)
            car_roi = frame[y:y+h, x:x+w]  # Region of interest for the detected car

            # Convert ROI to HSV color space
            hsv_roi = cv2.cvtColor(car_roi, cv2.COLOR_BGR2HSV)

            # Create a mask to filter out the white color
            white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
            
            # Count the number of white pixels in the mask
            white_pixels = cv2.countNonZero(white_mask)
            
            # If the white pixels exceed a threshold, count it as a white car
            if white_pixels > 2000:  # Adjust this threshold based on the size of white areas in a car
                white_car_count += 1
                # Draw bounding box and label it as "White Car"
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, "White Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Display the current count of white cars on the frame
    cv2.putText(frame, f"White Cars Count: {white_car_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Show the frame
    cv2.imshow("Frame", frame)
    cv2.imshow("Foreground Mask", fg_mask)  # For debugging motion detection

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"Total number of white cars: {white_car_count}")

Total number of white cars: 276


In [33]:
#task 4

import cv2

# Load the video
video_path = "C:\\Users\\DELL\\Downloads\\gettyimages-1995814914-640_adpp.mp4"
cap = cv2.VideoCapture(video_path)

# Define region of interest (ROI) near the entrance
roi_x, roi_y, roi_width, roi_height = 100, 200, 200, 150  # Adjust based on entrance position

# Background subtraction for motion detection
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50)

# Initialize counters for people entering and exiting
enter_count = 0
exit_count = 0

# Variables to hold movement direction
last_direction = None
direction_threshold = 30  # Minimum movement threshold for counting

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Define the ROI for detecting motion at the entrance
    roi = frame[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

    # Detect motion using background subtraction
    fg_mask = fgbg.apply(blurred_roi)
    _, fg_mask = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    # Find contours to identify moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Ignore small contours to avoid noise
        if cv2.contourArea(contour) < 500:
            continue

        # Draw bounding box around detected motion in the ROI
        x, y, w, h = cv2.boundingRect(contour)

        # Adjust the position of the bounding box vertically down
        roi_y += 10  # Increase the ROI's Y-coordinate to move down
        roi_height += 10  # Increase the height of the ROI

        # Ensure the ROI does not exceed the frame height
        roi_height = min(roi_height, frame.shape[0] - roi_y)

        # Draw the adjusted bounding box on the original frame
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x+roi_width, roi_y+roi_height), (0, 255, 0), 2)

        # Calculate movement direction (up for entering, down for exiting)
        if last_direction is None:
            last_direction = y
        else:
            direction = y - last_direction
            if abs(direction) > direction_threshold:
                if direction < 0:
                    enter_count += 1
                    print(f"Person entered, Total Entered: {enter_count}")
                elif direction > 0:
                    exit_count += 1
                    print(f"Person exited, Total Exited: {exit_count}")
                last_direction = y

    # Display the frame with updated ROI and motion highlighted
    cv2.putText(frame, f"Entered: {enter_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Exited: {exit_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow("Shop Entrance", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

print(f"Final count - Entered: {enter_count}, Exited: {exit_count}")

Person exited, Total Exited: 1
Person entered, Total Entered: 1
Person exited, Total Exited: 2
Person entered, Total Entered: 2
Final count - Entered: 2, Exited: 2


In [35]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Load the video
video_path = "C:\\Users\\DELL\\Downloads\\278ce34f-b588-43fc-a2ca-12083b012498 (1).mp4"
cap = cv2.VideoCapture(video_path)

# Background subtraction method (optional: use other methods like optical flow)
fgbg = cv2.createBackgroundSubtractorMOG2()

# Set frame rate for 1-minute intervals (assuming 30 FPS)
fps = cap.get(cv2.CAP_PROP_FPS)
interval_frames = int(fps * 60)  # 60 seconds worth of frames (1-minute interval)

# To store counts over time
people_counts = []
time_intervals = []

# To store frames for peak duration
peak_frames = []

# Initialize a deque to store frames for the current interval
frame_queue = deque(maxlen=interval_frames)

# Function to detect people in a frame (simplified using background subtraction)
def detect_people(frame):
    fgmask = fgbg.apply(frame)
    _, thresh = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    people_count = 0
    bounding_boxes = []
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filter by minimum area to avoid noise
            x, y, w, h = cv2.boundingRect(contour)
            bounding_boxes.append((x, y, w, h))
            people_count += 1
    return people_count, bounding_boxes

# Loop through the video frames
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for motion detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect people in the current frame
    people_count, bounding_boxes = detect_people(gray_frame)
    
    # Draw bounding boxes on the frame
    for (x, y, w, h) in bounding_boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box
    
    # Display the frame with bounding boxes
    cv2.imshow("Detected People", frame)

    # Add to current interval's frame queue
    frame_queue.append(people_count)

    # When the interval is full (1 minute), calculate total people count
    if len(frame_queue) == interval_frames:
        total_people = sum(frame_queue)
        people_counts.append(total_people)
        time_intervals.append(frame_idx / fps)  # Time in seconds for current interval
        
        # Update peak frames
        if total_people == max(people_counts):
            peak_frames = list(frame_queue)
    
    frame_idx += 1

    # Break on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display peak duration information
peak_time_interval = time_intervals[people_counts.index(max(people_counts))]
print(f"Peak Shopping Duration: {peak_time_interval} seconds")

# Display some of the frames from the peak interval
for idx, frame_count in enumerate(peak_frames):
    print(f"Peak Frame {idx+1}: {frame_count} people detected")

Peak Shopping Duration: 220.52665677546983 seconds
Peak Frame 1: 7 people detected
Peak Frame 2: 7 people detected
Peak Frame 3: 5 people detected
Peak Frame 4: 6 people detected
Peak Frame 5: 7 people detected
Peak Frame 6: 6 people detected
Peak Frame 7: 7 people detected
Peak Frame 8: 7 people detected
Peak Frame 9: 7 people detected
Peak Frame 10: 7 people detected
Peak Frame 11: 7 people detected
Peak Frame 12: 8 people detected
Peak Frame 13: 6 people detected
Peak Frame 14: 7 people detected
Peak Frame 15: 7 people detected
Peak Frame 16: 6 people detected
Peak Frame 17: 6 people detected
Peak Frame 18: 7 people detected
Peak Frame 19: 6 people detected
Peak Frame 20: 6 people detected
Peak Frame 21: 6 people detected
Peak Frame 22: 5 people detected
Peak Frame 23: 5 people detected
Peak Frame 24: 6 people detected
Peak Frame 25: 6 people detected
Peak Frame 26: 5 people detected
Peak Frame 27: 5 people detected
Peak Frame 28: 4 people detected
Peak Frame 29: 4 people detected
P

In [ ]:
"C:\Users\DELL\Downloads\gettyimages-1368-14-640_adpp.mp4"